# Introduction

This notebook allows you to run a simplified version of the ANTARESS workflow, for the following purpose:
- extracting from a CCF dataset the stellar lines occulted by a transiting planet, with spots present on the stellar surface (TBD)
- performing a Rossiter-McLaughlin Revolutions analysis of the dataset

The default version of this notebook is set up to process a mock dataset generated with ANTARESS (see `ANTARESS_nbook_mock-spots.ipynb`). To try it out, copy the directory `Valinor_Saved_data` from `Notebooks/Mock_system/` into the directory of your choice (named, for example, `Working_dir/`). Otherwise simply modify the settings to analyze a series of measured CCF fits files. Set `working_path` to the directory where you stored your copy of the `Valinor_Saved_data` directory, or where you would like your RMR analysis of an observed dataset to be stored.

To exploit the workflow to its full capabilities, run its executable with the [configuration files](https://obswww.unige.ch/~bourriev/antaress/doc/html/installation.html).

In [ ]:
import ANTARESS_nbook_bground
input_nbook = ANTARESS_nbook_bground.init('RMR')
input_nbook['working_path'] = '/Users/samsonmercier/Desktop/Work/Master/2023-2024/antaress/Ongoing/'

# System parameters

**Parameters of the host star**

Run this cell to define the host star. 
- `star_name` (string): Name of the star. Should not contain spaces/dashes.
- `Rs` (float): Radius of the star, in $R_\odot$.
- `vsini` (float): Projected stellar surface velocity, in km/s. The value given here is only used as guess for the fits, as it will eventually be derived from your analysis.
- `ld_u1` (float): First coefficient of the quadratic limb-darkening.
- `ld_u2` (float): Second coefficient of the quadratic limb-darkening.

In [ ]:
input_nbook['par'].update({
    'star_name' : 'Arda',
    'Rs'        : 0.9,
    'vsini'     : 3.,
    'ld_u1'     : 0.2,
    'ld_u2'     : 0.5
})
ANTARESS_nbook_bground.init_star(input_nbook)

**Parameters of transiting planet**

Run this cell to set the transiting planet on which the RM analysis is applied. 
- `planet_name` (string): Name of the transiting planet. Should not contain spaces/dashes.
- `period` (float): Orbital period, in days.
- `T0` (float): Mid-transit time, in ${\rm BJD}_{\rm TBD}$.
- `ecc` (float): Orbital eccentricity.
- `long_per` (float): Longitude of periastron, in degrees. If the orbit is circular, fill in `90.`.
- `Kstar` (float): RV semi-amplitude, in m/s.
- `aRs` (float): Scaled separation $a/R_\star$.
- `incl` (float): Orbital inclination, in degrees.
- `RpRs` (float): Planet-to-star radius ratio $R_{\rm p}/R_\star$.

In [ ]:
input_nbook['par'].update({
    'planet_name' : 'Valinor',
    'period'      : 3.8,
    'T0'          : 2457176.1,
    'ecc'         : 0.,
    'long_per'    : 90.,
    'Kstar'       : 38.1,
    'aRs'         : 12.0,
    'incl'        : 89.,
    'RpRs'        : 0.1
})
ANTARESS_nbook_bground.init_pl(input_nbook,'main')

**Parameters of additional planets in the system**

Run this cell to set additional planets, which will only contribute to the computation of the star Keplerian motion. 

The names and definitions of the parameters are the same as for the main planet (`aRs`, `incl`, `RpRs` are not required).

Copy the cell to add further planets

In [ ]:
input_nbook['par'].update({
    'planet_name' : 'Numenor',
    'period'      : 8.,
    'T0'          : 2457176.2,
    'ecc'         : 0.,
    'long_per'    : 90.,
    'Kstar'       : 20.
})
ANTARESS_nbook_bground.init_pl(input_nbook,'sec')

# Dataset properties

**Visit**

Run this cell to define the visit to be processed. It must overlap with the transit of the `transiting planet`.
- `instrument` (string): Instrument of the visit (e.g. `ESPRESSO`, `NIRPS_HE`...).
- `night` (string): Night of the visit. It is preferred to use the standard format, i.e. `YYYYMMDD`.
- `data_dir` (string): Path to the directory holding the observed .fits file (here, set to `mock` to retrieve the example dataset pre-generated with ANTARESS and stored in `Valinor_Saved_data/`)

In [ ]:
input_nbook['par'].update({
    'instrument'  : 'ESPRESSO',
    'night'       : '20230624',
    'data_dir'    : 'mock' 
})
ANTARESS_nbook_bground.add_vis(input_nbook)

**Radial velocity table**

Run this cell to set the radial velocity table over which CCFs are defined (in km/s, in the star rest frame).
- `x_start` (float): start of the table.
- `x_end` (float): end of the table.
- `dx` (float): step of the table.

In [ ]:
input_nbook['par'].update({
    'x_start' : -80.,
    'x_end'   :  80.,
    'dx'      :  0.8
})
ANTARESS_nbook_bground.set_mock_rv(input_nbook)

**Systemic velocity**

Run this cell to set the systemic radial velocity `gamma` (float, in km/s).

Run the notebook a first time with `gamma` = 0 km/s to determine the value specific to your chosen visit (`gamma` can vary slightly with the epoch and the instrument, and should thus be derived from the dataset itself rather than set to an external value). Go directly to the `Running ANTARESS` section and read the value of `gamma` printed in the log.

Then, set `gamma` to the derived value and proceed with the analysis.

In [ ]:
input_nbook['par']['gamma'] = 10.   
ANTARESS_nbook_bground.set_sysvel(input_nbook)

**Parameters of the main spot**

Run this cell to define the main spot. This spot will set the contrast for all other spots during the visit. The values given here for `lat`, `ang`, `Tc`, and `fctrst` are only used as guess for the fits, as they will eventually be derived from your analysis.
- `spot_name` (string): Name of the spot. Should not contain spaces/dashes.
- `lat` (float): Constant latitude of the spot, in degrees.
- `ang` (float): Half-angular size, in degrees.
- `Tc` (float): Time at which the spot is at longitude 0, in ${\rm BJD}_{\rm TBD}$.
- `fctrst` (float): Flux level of the spot surface, relative to the quiet surface of the star.
- `ld_spot_u1` (float): First coefficient of the spot's quadratic limb-darkening.
- `ld_spot_u2` (float): Second coefficient of the spot's quadratic limb-darkening.

In [ ]:
input_nbook['par'].update({
    'spot_name'    : 'Laurelin',
    'lat'          : 20.,
    'ang'          : 15.,
    'Tc'           : 2457176.0,
    'fctrst'       : 0.5,
    'ld_spot_u1'   : 0.2,
    'ld_spot_u2'   : 0.5
})
ANTARESS_nbook_bground.init_spot(input_nbook,'main')

**Parameters of additional spots**

Run this cell to set additional spots. 

The names and definitions of the parameters are the same as for the main spot (`fctrst`,`ld_spot_u1`, and `ld_spot_u1` are not required).

Copy the cell to add further spots.

In [ ]:
input_nbook['par'].update({
    'spot_name'    : 'Telperion',
    'lat'          : -10.,
    'ang'          : 20.,
    'Tc'           : 2457176.2,
})
ANTARESS_nbook_bground.init_spot(input_nbook,'sec')

# Processing

**Alignment in star rest frame**

Run this cell to align the disk-integrated CCFs in the star rest frame. 

CCFs are originally defined in the Sun barycentric rest frame. This module shifts them by the systemic rv you defined above, and by the Keplerian rv motion induced by the planets you set up for this system.

In [ ]:
ANTARESS_nbook_bground.align_prof(input_nbook)

**Flux scaling**

Run this cell to scale the disk-integrated CCFs to their correct relative flux level over the transit. 

The scaling depends on the limb-darkening coefficients and planet-to-star radius ratio you entered in the `System parameters`.

The flux value used to scale CCFs also accounts for the contribution of spots to the overall disk-integrated flux.

In [ ]:
ANTARESS_nbook_bground.flux_sc(input_nbook)
ANTARESS_nbook_bground.DImast_weight(input_nbook)

**Differential CCFs**

Run this cell to extract differential CCFs, which are the difference between the master out-of-transit CCF and the CCF in each exposure. As spots impact both the master out-of-transit and indivudal exposure CCFs, the differential CCFs during transit no longer correspond to the light coming from the planet-occulted region of the stellar photosphere. Instead, the differential CCFs are a complex mix of planet and spot contributions.

In [ ]:
ANTARESS_nbook_bground.extract_res(input_nbook)

# Analysis and plot display

Results from the fits you are going to perform are printed in the log of Section 6 below. For each fitted parameter, the workflow indicates its median and the 1-$\sigma$ errors and associated intervals from the Highest Density Intervals (when a MCMC has been used).

**Analysis of disk-integrated CCFs**

Run this cell to perform a Gaussian fit to each disk-integrated CCF. This will return the rv, FWHM, and contrast time-series of the average line from the entire stellar disk, which you can visualize below. This step is used to measure the systemic rv, to check that your Keplerian model matches the data, and to search for systematic trends in the line properties. 

Additional model and fit options are available through the configuration file. The full workflow will also allow you to correct for putative line trends.

In [ ]:
ANTARESS_nbook_bground.ana_prof(input_nbook,'DI')

**Plotting disk-integrated CCFs**

Run this cell to plot the disk-integrated CCF series with their best fits. Uncomment `y_range` to adjust the flux range of the CCFs (vertical axis).

In [ ]:
# input_nbook['par']['y_range'] = [2.1e5,3.6e5] 
ANTARESS_nbook_bground.plot_prof(input_nbook,'DI_prof')

**Plotting disk-integrated line properties**

Run this cell to plot the time-series of properties derived from the fits to disk-integrated CCFs. 

In [ ]:
ANTARESS_nbook_bground.plot_prop(input_nbook,'DI')

**Rossiter-McLaughlin Revolutions**

Run this cell to perform a RM Revolution fit to your dataset, accounting for spots. A joint model for the differential line profiles along the transit chord will be fitted to the exposures that you can select with `idx_in_fit`, based on your analysis from the previous step.

The model is controlled by the same parameters as for individual intrinsic properties (see `ANTARESS_nbook_RMR.ipynb`), so that you can define the model here in the same way using the parameters that you identified as relevant in the previous step. In addition, four properties must be specified for each spot (`lat`, `ang`, `Tc`, and `fctrst`) if we want to include them in the fit.

It is advised to run the fit through a MCMC, which you can set up after a preliminary run by checking the MCMC chains in `/Working_dir/planet_Saved_data/Joined_fits/ResProf/mcmc/`.

Best-fit values and associated uncertainties can be read in the log below, or retrieved with additional information in the file `/Working_dir/planet_Saved_data/Joined_fits/ResProf/mcmc/Outputs`.

The notebook assumes default properties for ANTARESS stellar grid, in particular, a simple Gaussian profile is assigned to each grid cell. Additional options are available through the configuration file. 

In [ ]:
#Exposures to be fitted
input_nbook['par']['idx_in_fit'] = range(1,20)

#Uncomment to use the MCMC
input_nbook['par']['fit_mode'] = 'mcmc'

#Uncomment to adjust the MCMC chains (default nwalkers = 3 x number of free parameters, nsteps = 5000, nburn = 1000)    
input_nbook['par']['mcmc_set']={'nwalkers':33,'nsteps':20,'nburn':8} 

#Model parameters
input_nbook['par']['mod_prop']={ 
    #Stellar parameters
    'veq':[1.,5.],
    'contrast_0':[0.5,0.8],    
    'FWHM_0':[1.,5.],
    
    #Planetary parameters
    'lambda':[0.,90.],
    
    #Spot parameters
    'lat_Laurelin':[-60.,40.],
    'Tc_Laurelin':[2457176.0 - 0.1, 2457176.0 + 0.3],
    'ang_Laurelin':[5.,55.],
    
    'lat_Telperion':[-40.,60.],
    'Tc_Telperion':[2457176.2 - 0.3, 2457176.2 + 0.1],
    'ang_Telperion':[5.,55.],
    
    'fctrst':[0.2,0.95],
}   

#Uncomment to set your own priors
input_nbook['par']['priors']={
    #Stellar parameters
    'veq':{'low':0.,'high':100.},  
    'contrast_0':{'low':0.,'high':1.},    
    'FWHM_0':{'low':0.,'high':20.},
    
    #Planetary parameters
    'lambda':{'low':-180.,'high':180.},
    
    #Spot parameters
    'lat_Laurelin':{'low':-90.,'high':90.},
    'Tc_Laurelin':{'low':2457176.0-1.,'high':2457176.0+1.},
    'ang_Laurelin':{'low':0.,'high':89.},
        
    'lat_Telperion':{'low':-90.,'high':90.},
    'Tc_Telperion':{'low':2457176.2-1.,'high':2457176.2+1.},
    'ang_Telperion':{'low':0.,'high':89.},
        
    'fctrst':{'low':0.,'high':1.}}    

#Set to True to run the fit, and to False to retrieve its results
input_nbook['par']['calc_fit'] = False

ANTARESS_nbook_bground.ana_jointprof(input_nbook,'Res')

**Plotting differential CCF maps**

Run this cell to plot the phase vs RV maps of your differential CCFs.

Adjust `v_range` to the flux range of your CCF dataset (corresponding to the color scale in the maps).

In [ ]:
# input_nbook['par']['v_range'] = [0.5,1.2]
ANTARESS_nbook_bground.plot_map(input_nbook,'Res_prof')

**Plotting model and residual differential CCF maps**

Run these cells to generate the model differential CCFs that correspond to the best RMR fit. Note that you first need to perform a RMR fit, and then run the notebook again to build the model and residuals (thus, leave the cell commented the first time you use the notebook).

In [ ]:
ANTARESS_nbook_bground.diff_prof_corr(input_nbook)

Run this cell to plot the phase vs RV maps of your model differential CCFs.

Adjust v_range to the flux range of your CCF dataset (corresponding to the color scale in the maps).

In [ ]:
# input_nbook['par']['v_range'] = [0.5,1.2]
ANTARESS_nbook_bground.plot_map(input_nbook,'BF_Res_prof')

Run this cell to plot the phase vs RV maps of the residuals between processed and model differential CCFs.

Adjust `v_range` to the residual flux range.

In [ ]:
# input_nbook['par']['v_range'] = [-0.01,0.01]
ANTARESS_nbook_bground.plot_map(input_nbook,'BF_Res_prof_re')

# Running ANTARESS

Run this cell to run the ANTARESS workflow.

In [ ]:
import os
os.chdir('../src')
from antaress.ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher
ANTARESS_launcher(working_path=input_nbook['working_path'], nbook_dic = input_nbook, exec_comm=False)

# Plot display

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

Run the cells below to show saved plots.

**Disk-integrated CCF series**

This plot shows the series of disk-integrated CCFs (blue profiles) with their best-fit Gaussian models (dashed black profiles), from which are derived the properties showed in the next plot.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Indiv/Data/CCF/phase.gif')

**Disk-integrated line properties**

The three plots below show the time-series of RVs, contrast, and FWHM derived from the fits to the disk-integrated CCFs. Transit contacts are shown as dashed black lines, and in-transit exposures are flagged with empty symbols.

If spots were absent, RVs outside of transit would trace the Keplerian motion of your star with respect to the solar system barycenter (the Keplerian model defined by your inputs for the star and planets is plotted as a solid blue curve), while RVs inside of the transit would display the typical RM signature.

In the presence of spots, disk-integrated CCFs present additional distortions which bias the retrieved line properties. In this case, it is recommended to apply the RM Revolution technique on the differential CCF series to retrieve the system's properties.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/ctrst_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/FWHM_phase.png')

**Differential CCF series**

This plot shows the series of differential CCFs (blue profiles), from which are derived the properties showed in the next plots.

In [ ]:
Image(filename = input_nbook['plot_path']+'Res_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Indiv/Data/CCF/phase.gif')

**Differential CCF Maps**

These plots shows the series of processed, modeled and residual differential CCFs. In all three plots, the series is shown in phase-RV space, colored with flux.

In [ ]:
Image(filename = input_nbook['plot_path']+'Res_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Map/Data/CCF/iord0.png', width=800)

In [ ]:
if input_nbook['par']['diff_prof_corr']:
    Image(filename = input_nbook['plot_path']+'Res_data/ESPRESSO_20230624_Map/Data/CCF/iord0BestFit.png',width=800)

In [ ]:
if input_nbook['par']['diff_prof_corr']:
    Image(filename = input_nbook['plot_path']+'Res_data/ESPRESSO_20230624_Map/Data/CCF/iord0BestFitDifferential.png', width=800)

# Results comparison for the example mock dataset

If you ran the successive steps with relevant settings, you should have retrieved best-fit values from the RM Revolutions analysis that match the properties used to generate the mock dataset:

**Star**
- a projected stellar surface velocity `vsini` = 3 km/s
- an intrinsic stellar line `contrast` = 0.7
- an intrinsic stellar line `FWHM` = 2.5 km/s 

**Planet**
- a sky-projected spin-orbit angle `lambda` = 35 degrees

**Spots**

- a flux level `fctrst` = 0.5

*Laurelin*

- a latitude of `lat` = 20 degrees
- a half-angular size of `ang` = 15 degrees
- a spot-crossing time `Tc` = 2457176.0 BJD

*Telperion*
- a latitude of `lat` = -10 degrees
- a half-angular size of `ang` = 20 degrees
- a spot-crossing time `Tc` = 2457176.2 BJD